# 7.7.6 Taller: Clasificación

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

In [7]:
import sys
sys.path.append("../../../../../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Analitica predictiva'
data = make_dir_line(modality, project)

raw = data('raw')



## 7.7.6.1 Descripción del problema



## 7.7.6.2 Carga de los datos

In [8]:
df = pd.read_excel(
    raw / "demanda_comercial.xlsx",
    sheet_name="DemandaComercial",
)

df.head(10)

,Fecha,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,2017-01-01,6090271.96,5887031.28,5708693.89,5538133.02,5389774.53,5288199.94,4851217.85,4876162.18,5073813.27,...,5745568.87,5647807.40,5609297.84,5690105.56,6761038.26,7077861.59,6979468.90,6695312.12,6289107.72,5794545.98
1,2017-01-02,5557603.42,5361012.17,5239119.86,5163721.70,5249539.75,5438852.95,5566947.57,6055220.80,6588005.66,...,7610559.73,7584026.97,7421194.49,7317194.26,8182697.67,8482127.69,8251009.75,7836278.95,7221234.85,6660691.64
2,2017-01-03,6160676.02,5924880.04,5764416.93,5685832.28,5778222.13,5985840.29,6083907.36,6545100.21,7092804.54,...,8148614.93,8117168.00,7930900.03,7783762.28,8653467.38,8835882.88,8562448.23,8095841.79,7443299.51,6840297.15
3,2017-01-04,6321851.20,6092135.24,5905390.85,5827867.82,5925730.02,6182279.89,6276759.96,6737660.53,7256819.19,...,8164513.51,8132006.92,7962749.39,7841533.59,8700506.07,8860255.66,8611085.76,8147451.75,7471240.82,6838250.32
4,2017-01-05,6395397.96,6162409.22,6001153.67,5918673.26,6005924.21,6225027.53,6322923.78,6739120.44,7307847.42,...,8144209.65,8136193.07,8022918.36,7927405.07,8718181.89,8832234.87,8579992.47,8063978.64,7406615.66,6829864.57
5,2017-01-06,6381819.62,6131392.34,5995648.54,5926665.13,6016864.72,6294425.78,6434787.53,6882054.00,7365946.07,...,8240411.86,8173415.39,7996806.30,7822605.04,8675548.79,8846841.24,8592264.60,8136250.38,7518243.88,6921639.21
6,2017-01-07,6525895.31,6240836.72,6090880.04,5990958.51,6031128.61,6153095.75,6153441.16,6494933.30,7032859.86,...,7541365.53,7431992.46,7301062.46,7354996.31,8254006.56,8462231.18,8247014.72,7846633.93,7263016.84,6748022.10
7,2017-01-08,6326836.49,6094599.00,5894932.99,5767066.60,5723343.19,5732869.09,5517452.94,5695553.38,6006775.90,...,6530022.48,6424695.70,6390754.63,6548894.08,7552952.28,7754913.10,7563722.46,7188256.16,6705884.31,6238894.75
8,2017-01-09,5917367.06,5725053.67,5567058.06,5496669.03,5532239.19,5566907.36,5397204.37,5600906.11,5965768.21,...,6659606.88,6544740.37,6509866.66,6712253.52,7703462.32,7979736.68,7716144.47,7297017.51,6795361.42,6321478.25
9,2017-01-10,5956092.23,5712085.93,5606050.44,5595323.09,5770049.55,6103857.89,6296790.66,6776968.47,7381221.79,...,8317358.60,8286351.48,8191637.05,8053310.11,8803188.92,8925547.21,8563960.40,8036507.99,7340419.13,6664161.76


## 7.7.5.3 Análisis exploratorio

In [10]:
#
# Número de registros
#
len(df)

30000

In [11]:
#
# Valores diferentes de NA
#
df.gender.value_counts()

gender
F    22054
M     5222
Name: count, dtype: int64

In [12]:
#
# Número de registros con NA
#
df.gender.isna().sum()

2724

In [14]:
#
# Se analizan los rangos de las variables
# para determinar si hay datos por fuera de sus
# rangos válidos. La variable `edad` contiene
# datos por fuera de la población de interés
#
df.age.describe()

count    24914.000000
mean        17.993950
std          7.858054
min          3.086000
25%         16.312000
50%         17.287000
75%         18.259000
max        106.927000
Name: age, dtype: float64

In [15]:
# cantidad de nulos en la columna age
df.age.isnull().sum()

5086

In [17]:
#
# Se seleccionan las personas entre 13 y 20 años y
# se descartan las demás observaciones
#
df['age'] = df['age'].map(lambda x: np.nan if x <  13 else x)
df['age'] = df['age'].map(lambda x: np.nan if x >= 20 else x)

#
# Se verifica la variable edad en los registros de las
# personas en la población de interés.
#
df.age.describe()

count    24477.000000
mean        17.252429
std          1.157465
min         13.027000
25%         16.304000
50%         17.265000
75%         18.220000
max         19.995000
Name: age, dtype: float64

In [19]:
# cantidad de nulos en la columna age
df.age.isnull().sum()

5523

In [23]:
#
# Edad sin tener en cuenta los datos faltantes
#
df.age.mean()

17.252428933284307

In [18]:
#
# Se crean nuevas variables numéricas a partir
# de información categórica
#
df['female'] = df['gender'].map(lambda x: 1 if x == 'F' else 0)
df['no_gender'] = df['gender'].map(lambda x: 1 if pd.isnull(x) else 0)

# Cantidad de hombres y mujeres en la muestra.
df.gender.value_counts()

gender
F    22054
M     5222
Name: count, dtype: int64

In [20]:
#
# Número de registros con NA
#
df.gender.isna().sum()

2724

In [21]:
#
# Cantidad de hombres y mujeres en la muestra
# 0=Male, 1=Female
#
df.female.value_counts()

female
1    22054
0     7946
Name: count, dtype: int64

In [22]:
#
# Cantidad de patrones para los que se
# conoce y no se conoce el genero
#
df.no_gender.value_counts()

no_gender
0    27276
1     2724
Name: count, dtype: int64

In [24]:
#
# Se calcula una tabla para determinar
# la edad promedio por año de graduación
#
df.groupby("gradyear")['age'].mean()

gradyear
2006    18.655858
2007    17.706172
2008    16.767701
2009    15.819573
Name: age, dtype: float64

In [25]:
#
# Los valores faltantes se llenan con el promedio
# de acuerdo con el año de graduación
#
ave_age = df.groupby("gradyear")['age'].mean()
df.age = [x if not pd.isnull(x) else ave_age[y] for x, y in zip(df.age, df.gradyear)]
df.age.describe()

count    30000.000000
mean        17.237326
std          1.141821
min         13.027000
25%         16.282000
50%         17.238000
75%         18.212000
max         19.995000
Name: age, dtype: float64

## 7.7.5.4 Entrenamiento del modelo

In [26]:
#
# Se separa la información de las palabras relacionadas
# con los intereses del resto de la información
##
interests = df.iloc[:,4:].copy()
interests.head()

,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,...,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs,female,no_gender
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,1,0,1


In [28]:
#
# Se escala para eliminar problemas asociados
# a la medida de los datos.
#
scaler = MinMaxScaler()
interests_z = scaler.fit_transform(interests)
interests_z

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06666667, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.06666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.06666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08333333, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [30]:
#
# Se usa el algoritmo para determinar los centros de 5 grupos
#
df_clusters = KMeans(n_clusters=5, random_state=1).fit(interests_z)

## 7.7.5.5 Evaluación del modelo

In [31]:
#
# Número de patrones asignados a cada cluster
#
(pd.DataFrame(df_clusters.predict(interests_z)))[0].value_counts()

0
3    16463
0     5222
4     3111
2     2724
1     2480
Name: count, dtype: int64

In [32]:
#
# Frecuencia de las palabras en cada cluster
#
df_clusters.cluster_centers_

array([[ 1.31415167e-02,  2.98991446e-02,  7.78047293e-03,
         5.40698853e-04,  2.47578924e-03,  2.57595038e-03,
         9.57487553e-04,  1.69714669e-02,  6.88114388e-03,
         1.66443253e-02,  3.95761522e-03,  1.40767468e-03,
         4.57466275e-03,  5.18958254e-03,  2.26850897e-03,
         6.85561088e-03,  5.32130961e-03,  4.45666934e-03,
         1.08196093e-02,  1.20461053e-02,  5.41283470e-03,
         4.44796490e-03,  4.16826248e-03,  2.48946764e-03,
         4.88059871e-03,  3.42567769e-03,  7.43736509e-05,
         1.13302694e-02,  4.43926047e-03,  9.02432018e-03,
         3.63845270e-03,  2.84852547e-03,  8.10382647e-03,
         7.44104612e-03,  9.40731520e-03,  3.44695519e-03,
        -1.18793864e-14, -1.30451205e-15],
       [ 1.71845574e-02,  2.30024213e-02,  1.04176008e-02,
         1.66642928e-02,  1.43837196e-02,  7.12072691e-03,
         1.07613667e-02,  7.91969330e-03,  7.37153619e-03,
         1.93704600e-02,  3.37637880e-02,  4.74703708e-03,
         1.30

Actividad.— Haga un heatmap para la interpretación de los grupos.

Actividad.— Cuál de los clusters anteriores tiene más interés en los deportes?

Actividad.— Cuál es el cluster de las princesas?

Actividad.— Cuál cluster representa las personas que no hablan o no han posteado sobre sus intereses?

## 7.7.5.6 Análisis del modelo

In [33]:
#
# Se asigna a cada ejemplo de los datos
# el cluster al que pertenece
#
df["cluster"] = df_clusters.predict(interests_z).tolist()

#
# clusters a los que pertenecen los primeros cinco patrones
#
df[["cluster", "gender", "age", "friends"]].head(5)

,cluster,gender,age,friends
0,0,M,18.982,7
1,4,F,18.801,0
2,0,M,18.335,69
3,3,F,18.875,0
4,2,NaN,18.995,10


In [34]:
#
# Características demográficas de los clusters.
# Edad por cluster.
#
df.groupby("cluster")['age'].mean()

cluster
0    17.432631
1    17.071663
2    17.269795
3    17.229713
4    17.053415
Name: age, dtype: float64

In [35]:
#
# Promedio de mujeres por cluster
#

#
# cluster   female
#       1 0.8381171
#       2 0.7250000
#       3 0.8378198
#       4 0.8027079
#       5 0.6994515
#
df.groupby("cluster")['female'].mean()

cluster
0    0.0
1    1.0
2    0.0
3    1.0
4    1.0
Name: female, dtype: float64

In [36]:
for a,b in df.groupby("cluster"):
    print(b['female'].sum())

0
2480
0
16463
3111


In [37]:
#
# Cantidad promedio de amigos por cluster
#
df.groupby("cluster")['friends'].mean()

cluster
0    24.926465
1    33.369758
2    26.139134
3    30.261374
4    39.558020
Name: friends, dtype: float64

Actividad.— Al analizar las distintas variables presentadas, qué puede inferir de cada cluster?

Actividad.— Repita este ejercicio usando clustering jerárquico.

In [8]:
print('ok_')

ok_
